In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
import os
from multiprocessing import cpu_count
from typing import List, Iterable, Set, Optional, Dict, Tuple

import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np

# from model import get_m1_model, get_m1_bigger_model, get_m1_smaller_model
from model import MODEL_GETTERS_DICT
from ns_tokenizers import CharLevelTokenizerv2, KeyboardTokenizerv1
from ns_tokenizers import ALL_CYRILLIC_LETTERS_ALPHABET_ORD
from dataset import CurveDataset, CurveDatasetSubset
from word_generators_v2 import GreedyGenerator, BeamGenerator, WordGenerator
from predict_v2 import Predictor
from metrics import get_mmr
from aggregate_predictions import (separate_out_vocab_all_crvs,
                                   append_preds,
                                   create_submission,
                                   merge_default_and_extra_preds)
from feature_extractors import weights_function_v1
from feature_extractors import get_val_transform
from grid_processing_utils import get_grid

In [4]:
IN_KAGGLE = False

if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/yandex-cup-playground"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_ROOT = "../data/trained_models_for_final_submit"

In [5]:
VAL_PATH = os.path.join(DATA_ROOT, "valid__in_train_format.jsonl")
TEST_PATH = os.path.join(DATA_ROOT, "test.jsonl")

VOCAB_PATH = os.path.join(DATA_ROOT, "voc.txt")
G_NAME_TO_GRID_PATH = os.path.join(DATA_ROOT, "gridname_to_grid.json")

In [79]:
kb_tokenizer = KeyboardTokenizerv1()
char_tokenizer = CharLevelTokenizerv2(VOCAB_PATH)

data_paths = [
    VAL_PATH,
    TEST_PATH
]

# takes almost no time
dist_transform_v1 = get_val_transform(
    gridname_to_grid_path=G_NAME_TO_GRID_PATH,
    grid_names=('default', 'extra'),
    transform_name="traj_feats_and_distances",
    char_tokenizer=char_tokenizer,
    dist_weights_func=weights_function_v1,
    include_time=False,
    include_velocities=True,
    include_accelerations=True
)

# takes a lot of time
kb_transform = get_val_transform(
    gridname_to_grid_path=G_NAME_TO_GRID_PATH,
    grid_names=('default', 'extra'),
    transform_name="traj_feats_and_nearest_key",
    char_tokenizer=char_tokenizer,
    uniform_noise_range=0,
    ds_paths_list=data_paths,
    totals = [10_000, 10_000],
    include_time=False,
    include_velocities=True,
    include_accelerations=True
)

Accumulating out-of-bounds coordinates...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

augmenting gname_to_out_of_bounds


In [115]:
val_dataset = CurveDataset(
    data_path=VAL_PATH,
    store_gnames=True,
    init_transform=None,
    # get_item_transform=kb_transform,
    get_item_transform=dist_transform_v1,
)

test_dataset = CurveDataset(
    data_path=TEST_PATH,
    store_gnames=True,
    init_transform=None,
    # get_item_transform=kb_transform,
    get_item_transform=dist_transform_v1,
)

10000it [00:00, 11101.07it/s]
10000it [00:00, 13174.88it/s]


In [116]:
val_default_dataset = CurveDatasetSubset(val_dataset, "default")
val_extra_dataset = CurveDatasetSubset(val_dataset, "extra")

test_default_dataset = CurveDatasetSubset(test_dataset, "default")
test_extra_dataset = CurveDatasetSubset(test_dataset, "extra")

In [64]:
def get_targets(dataset: CurveDataset, 
                char_tokenizer: CharLevelTokenizerv2) -> List[str]:
    targets = []
    for _, target_tokens in dataset:
        target = char_tokenizer.decode(target_tokens[:-1])
        targets.append(target)
    return targets

In [65]:
def get_vocab_set(vocab_path: str):
    with open(vocab_path, 'r', encoding = "utf-8") as f:
        return set(f.read().splitlines())

In [66]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [81]:
MODEL_GETTERS_DICT.keys()

dict_keys(['m1', 'm1_bigger', 'm1_smaller', 'transformer_m1_bigger', 'transformer_bb_model', 'weighted_transformer_bigger', 'v2_weighted_transformer_bigger', 'v2_nearest_transformer_bigger', 'v3_weighted_and_traj_transformer_bigger', 'v3_nearest_and_traj_transformer_bigger', 'v3_nearest_only_transformer_bigger', 'v3_trainable_gaussian_weights_and_traj_transformer_bigger'])

In [98]:
grid_name = "default"
model_getter = MODEL_GETTERS_DICT['v3_weighted_and_traj_transformer_bigger']
weights_path = r"..\results\models_for_debug\weighted_transformer_bigger-default--epoch=60-val_loss=0.442-val_word_level_accuracy=0.875.pt"

# model_getter = MODEL_GETTERS_DICT['v3_nearest_and_traj_transformer_bigger']
# weights_path = r"..\results\models_for_debug\my_features_1\v3_nearest_and_traj_transformer_bigger-default--epoch=32-val_loss=0.441-val_word_level_accuracy=0.864.pt"

model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, char_tokenizer, device)}

In [101]:
def remove_suffix(s: str, suffix: str) -> str:
    if s.endswith(suffix):
        return s[:-len(suffix)]
    return s

In [102]:
n_classes = 35

greedy_generator__no_vocab = GreedyGenerator(model, char_tokenizer, device)
beam_generator__no_vocab = BeamGenerator(model, char_tokenizer, device)

greedy_generator_with_vocab = GreedyGenerator(model, char_tokenizer, device, vocab_set, max_token_id=n_classes-1)
beam_generator__with_vocab = BeamGenerator(model, char_tokenizer, device, vocab=vocab_set, max_token_id=n_classes-1)

In [103]:
def denormalize_beamsearch(score, pred_len, normalization_factor):
    return score * (pred_len + 1)**normalization_factor

def beamsearch_score_to_prob(score, pred_len, normalization_factor):   
    return np.exp(denormalize_beamsearch(-score, pred_len, normalization_factor))

In [104]:
from typing import Callable

MAX_WORD_LEN = 35

def predict(word_generator: WordGenerator, dataset, n_hypotheses, scores_to_prob: Callable, 
            max_steps_n: int = MAX_WORD_LEN, verbose = True, 
            n_examples = None, generator_call_kwargs = None, ) -> List[List[Tuple[str, float]]]:
    
    curve_id_to_hypotheses = []

    n_examples = n_examples or len(dataset)
    generator_call_kwargs = {} if generator_call_kwargs is None else generator_call_kwargs
    

    if verbose:
        print(("{:<22}" + "{:<29}" * n_hypotheses).format("target", *[f"pred{i}" for i in range(1, n_hypotheses+1)]))
        print("-"*(15+30*n_hypotheses))

    for i, data in enumerate(val_default_dataset):

        (encoder_in, dec_in), target = data

        scores_and_preds_full = word_generator(encoder_in, max_steps_n = max_steps_n, 
            **generator_call_kwargs)

        preds_and_probs_full = [(pred, scores_to_prob(pred, score)) 
                                for score, pred in scores_and_preds_full]

        curve_id_to_hypotheses.append(preds_and_probs_full)

        true_label = char_tokenizer.decode(target[:-1])

        flat_preds_and_scores = (item for pair in preds_and_probs_full[:n_hypotheses] for item in pair)
        if verbose:
            print(("{:<15}   |   " + "{:<16}{:.4f}   |   " *n_hypotheses ).format(true_label, *flat_preds_and_scores))

        if i >= n_examples:
            return curve_id_to_hypotheses
    
    return curve_id_to_hypotheses

In [119]:
curve_id_to_hypotheses = predict(
    greedy_generator_with_vocab, val_dataset, n_hypotheses=1, 
    scores_to_prob=lambda pred, score: np.exp(-score), n_examples=15)

target                pred1                        
---------------------------------------------
на                |   на              0.8950   |   
все               |   все             0.8705   |   
добрый            |   добрый          0.8608   |   
девочка           |   девочка         0.8550   |   
сказала           |   сказала         0.8633   |   
скинь             |   скинь           0.8786   |   
геев              |   геев            0.8842   |   
тобой             |   тобой           0.8880   |   
была              |   баса            0.4085   |   
да                |   да              0.8957   |   
муж               |   муж             0.8350   |   
щас               |   щас             0.9516   |   
она               |   она             0.9011   |   
проблема          |   проблема        0.8468   |   
билайн            |   билайн          0.8468   |   
уже               |   уже             0.9064   |   


In [120]:
curve_id_to_hypotheses = predict(
    greedy_generator__no_vocab, val_dataset, n_hypotheses=1, 
    scores_to_prob=lambda pred, score: np.exp(-score), n_examples=15)

target                pred1                        
---------------------------------------------
на                |   на              0.8729   |   
все               |   все             0.8297   |   
добрый            |   добрый          0.7269   |   
девочка           |   девочка         0.6857   |   
сказала           |   сказала         0.7000   |   
скинь             |   скинь           0.7596   |   
геев              |   геев            0.7808   |   
тобой             |   тобой           0.7647   |   
была              |   баса            0.3738   |   
да                |   да              0.8705   |   
муж               |   муж             0.7908   |   
щас               |   щас             0.8447   |   
она               |   она             0.8367   |   
проблема          |   проблема        0.6630   |   
билайн            |   билайн          0.7044   |   
уже               |   уже             0.8313   |   


In [122]:
n_examples = 15
normalization_factor = 0.5
beamsize = 6

curve_id_to_hypotheses = predict(
    beam_generator__with_vocab, val_dataset, n_hypotheses=4, n_examples=n_examples,
    generator_call_kwargs={'normalization_factor': normalization_factor, 'beamsize': beamsize}, 
    scores_to_prob =lambda pred, score: beamsearch_score_to_prob(score, len(pred) + 1, normalization_factor))

target                pred1                        pred2                        pred3                        pred4                        
---------------------------------------------------------------------------------------------------------------------------------------
на                |   на              0.8950   |   нан             0.0014   |   нам             0.0013   |   нас             0.0013   |   
все               |   все             0.8705   |   всенародная     0.0002   |   всенародно      0.0001   |   всенародную     0.0001   |   
добрый            |   добрый          0.8608   |   добрый-добрый   0.0011   |   добрым          0.0039   |   добрые          0.0019   |   
девочка           |   девочка         0.8550   |   девочка-волшебница0.0002   |   девочки         0.0029   |   девочку         0.0025   |   
сказала           |   сказала         0.8633   |   сказал-сделал   0.0008   |   сказал          0.0037   |   сказали         0.0026   |   
скинь             |   скинь 

In [123]:
n_examples = 15
normalization_factor = 0.5

curve_id_to_hypotheses = predict(
    beam_generator__no_vocab, val_dataset, n_hypotheses=4, n_examples=n_examples,
    generator_call_kwargs={'normalization_factor': normalization_factor, 'beamsize': beamsize}, 
    scores_to_prob =lambda pred, score: beamsearch_score_to_prob(score, len(pred) + 1, normalization_factor))

target                pred1                        pred2                        pred3                        pred4                        
---------------------------------------------------------------------------------------------------------------------------------------
на                |   на              0.8729   |   нан             0.0014   |   нак             0.0013   |   наз             0.0012   |   
все               |   все             0.8297   |   всем            0.0015   |   вссе            0.0012   |   всен            0.0012   |   
добрый            |   добрый          0.7269   |   добрым          0.0032   |   добрыйо         0.0011   |   добрыйи         0.0010   |   
девочка           |   девочка         0.6857   |   девочки         0.0023   |   девочку         0.0020   |   девочкам        0.0013   |   
сказала           |   сказала         0.7000   |   сказал          0.0031   |   сказали         0.0021   |   сказалась       0.0009   |   
скинь             |   скинь   

In [97]:
print(curve_id_to_hypotheses[0][:6])
print()
print(curve_id_to_hypotheses[1][:6])

[('на', 0.8729147558351656), ('нас', 0.001725468331042225), ('неа', 0.001583207265670061), ('нам', 0.0012425854680964096), ('нав', 0.0011460283846146304), ('наа', 0.0011295282480705778)]

[('все', 0.8300481755078998), ('всем', 0.0012949032335612689), ('всен', 0.001187697357643541), ('всеб', 0.0011859830734336145), ('всех', 0.0011663816457048192), ('всек', 0.0011443660631516442)]


---------------
Как и ожидается, после денормализации бимсерча вероятности не такие же как в greedy

Нужно отметить, что в общем случае должно быть так:
* greedy_search_no_vocab__results == beamsearch__no_vocab__results
* greedy_search_with_voc__results == beamsearch__with_voc__results
* greedy_search_with_voc__results != beamsearch__no_vocab__results
* greedy_search__no_vocab__results != beamsearch__with_voc__results



Некоторые заметки:
1. Иногда вероятность оказывается немонотонной. Это именно из-за того, что мы убрали нормализацию бимсерча. То есть убрали штраф за краткость, а при ранжировании он был
2. Даже если beamsize = n_classes, beamsearch не оценивает явно вероятность всех слов, потому что из-за over confidence вероятности некоторых возможных токенов оказываются нулевыми и эта ветка обрывается
---------------

# Evaluate models separately and as a pair

In [126]:
print(cpu_count())

4


In [128]:
val_default_targets = get_targets(val_default_dataset, char_tokenizer)
val_extra_targets = get_targets(val_extra_dataset, char_tokenizer)

In [ ]:
default_predictions = weights_to_raw_predictions(
    grid_name = "default",
    model_getter=get_m1_bigger_model,
    weights_path = os.path.join(MODELS_ROOT, "m1_bigger/m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt"),
    char_tokenizer=char_tokenizer,
    dataset=val_default_dataset,
    generator_ctor=GreedyGenerator,
    n_workers=4
)

In [135]:
default_MMR =  get_mmr(default_predictions, val_default_targets)
default_MMR

0.8531223449447749

In [136]:
default_predictions_best_bigger = default_predictions

In [137]:
default_predictions_best_bigger_clean, _ = separate_out_vocab_all_crvs(default_predictions_best_bigger, vocab_set)

In [138]:
sum(bool(el) for el in default_predictions_best_bigger_clean)

8784

In [182]:
default_predictions = weights_to_raw_predictions(
    grid_name = "default",
    model_getter=get_m1_smaller_model,
    weights_path = os.path.join(MODELS_ROOT, "m1_smaller/m1_smaller_v2_2023_11_11_17_43_35_0_33179_default_l2_1e_05_ls0_02.pt"),
    char_tokenizer=char_tokenizer,
    dataset=val_default_dataset,
    generator_ctor=GreedyGenerator,
    n_workers=4
)

100%|██████████| 9416/9416 [04:08<00:00, 37.90it/s]


In [183]:
default_predictions_clean, _ = separate_out_vocab_all_crvs(default_predictions, vocab_set)

In [184]:
sum(bool(el) for el in default_predictions_clean)

8449

In [185]:
default_MMR =  get_mmr(default_predictions, val_default_targets)
default_MMR

0.8221112999150383

In [110]:
# grid_name = "default"
# model_getter = get_m1_bigger_model
# weights_path = os.path.join(MODELS_ROOT, "m1_bigger/m1_bigger_v2__2023_11_11__13_17_50__0.13845_default_l2_0_ls0_switch_0.pt")
# model = model_getter(device, weights_path)
# grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, char_tokenizer, device)}

In [111]:
# default_predictions = predict_greedy_raw_multiproc(val_default_dataset,
#                                                     grid_name_to_greedy_generator,
#                                                     num_workers=4)

100%|██████████| 9416/9416 [05:02<00:00, 31.10it/s]


In [29]:
grid_name = "extra"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, char_tokenizer, device)}

In [31]:
extra_predictions = predict_raw_mp(val_extra_dataset,
                                   grid_name_to_greedy_generator,
                                   num_workers=4)

100%|██████████| 584/584 [00:18<00:00, 31.60it/s]


In [33]:
extra_MMR = get_mmr(extra_predictions, val_extra_targets)
extra_MMR

0.851027397260274

In [41]:
all_preds = merge_default_and_extra_preds(default_predictions, extra_predictions, val_default_dataset.grid_name_idxs, val_extra_dataset.grid_name_idxs)

In [40]:
all_targets = None
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding='utf-8') as f:
    all_targets = f.read().splitlines() 

In [46]:
full_MMR = get_mmr(all_preds, all_targets)
full_MMR

0.8512

In [365]:
max(len(el) for el in all_targets)

19

In [ ]:
{"m1_v2/best_model__2023_11_09__10_36_02__0.14229_default_switch_2_try_2.pt": 0.8512107051826678,
 "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt": 0.851027397260274,
 
 "m1_bigger/m1_bigger_v2__2023_11_10__13_38_32__0.50552_default_l2_5e-05_ls0.045_switch_0.pt": 0.810429056924384,
 "m1_bigger/m1_bigger_v2__2023_11_10__16_36_38__0.49848_default_l2_5e-05_ls0.045_switch_0.pt": 0.818500424808836,
 "m1_bigger/m1_bigger_v2__2023_11_10__21_51_01__0.49382_default_l2_5e-05_ls0.045_switch_0.pt": 0.8210492778249787,
 
 "m1_bigger/m1_bigger_v2__2023_11_11__13_17_50__0.13845_default_l2_0_ls0_switch_0.pt": 0.8512107051826678,
 "m1_bigger/m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt": 0.8531223449447749,
 
 "m1_smaller/m1_smaller_v2_2023_11_11_17_43_35_0_33179_default_l2_1e_05_ls0_02.pt": 0.8221112999150383}

# Let's create a greedy submission

In [73]:
from typing import Set, List, Tuple

In [ ]:
grid_name = "default"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, char_tokenizer, device)}
default_test_predictions = predict_raw_mp(test_default_dataset,
                                          grid_name_to_greedy_generator,
                                          num_workers=3)

In [15]:
grid_name = "extra"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, char_tokenizer, device)}
extra_test_predictions = predict_raw_mp(test_extra_dataset,
                                        grid_name_to_greedy_generator,
                                        num_workers=3)

100%|██████████| 627/627 [00:20<00:00, 30.18it/s]


In [66]:
all_test_preds = merge_default_and_extra_preds(default_test_predictions, extra_test_predictions, test_default_dataset.grid_name_idxs, test_extra_dataset.grid_name_idxs)

In [69]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [91]:
clean_test_preds, invalid_test_preds = separate_out_vocab_all_crvs(all_test_preds, vocab_set)

In [92]:
clean_test_preds[:10]

[['на'],
 ['что'],
 ['опоздания'],
 ['сколько'],
 [],
 ['не'],
 ['как'],
 ['садовод'],
 ['заметил'],
 ['ваги']]

In [81]:
augment_list = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    augment_lines = f.read().splitlines()
augment_list = [line.split(",") for line in augment_lines]

In [96]:
augmented_test_preds = append_preds(clean_test_preds, augment_list, limit = 4)

In [97]:
augmented_test_preds[:10]

[['на', 'неа', 'на', 'ненка'],
 ['что', 'часто', 'частого', 'чисто'],
 ['опоздания', 'опоздания', 'опозданиям', 'оприходования'],
 ['сколько', 'сколько', 'сокольского', 'свердловского'],
 ['дремать', 'дописать', 'донимать', 'дюрренматт'],
 ['не', 'неук', 'нк', 'ненка'],
 ['как', 'как', 'капак', 'капе'],
 ['садовод', 'спародировал', 'садовод', 'сурдоперевод'],
 ['заметил', 'знаменито', 'знаменитого', 'замерил'],
 ['ваги', 'ваенги', 'венгрии', 'ванги']]

In [98]:
submission_name = "m1_v2__0.14229_deault__0.14301_extra__greedy.csv"
out_path = rf"..\data\submissions\{submission_name}"
create_submission(augmented_test_preds, out_path)

# BeamSearch

In [ ]:
from typing import Tuple, List
def remove_beamsearch_probs(preds: List[List[Tuple[float, str]]]) -> List[List[str]]:
    new_preds = []
    for pred_line in preds:
        new_preds_line = []
        for _, word in pred_line:
            new_preds_line.append(word)
        new_preds.append(new_preds_line)
    return new_preds

In [ ]:
def patch_wrong_prediction_shape(prediciton):
    return [pred_el[0] for pred_el in prediciton]

## Beamsearch Evaluation

In [372]:
MAX_VAL_WORD_LEN = max(len(el) for el in all_targets)

generator_kwargs = {
    'max_steps_n': MAX_VAL_WORD_LEN+1,
    'return_hypotheses_n': 7,
    'beamsize': 6,
    'normalization_factor': 0.5,
    'verbose': False
}

In [373]:
grid_name_to_val_dataset = {
    'default': val_default_dataset,
    'extra': val_extra_dataset
}

In [374]:
import pickle

bs_params = [
    ("default", get_m1_bigger_model, "m1_bigger/m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt"),
]


for grid_name, model_getter, weights_f_name in bs_params:

    bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                f"{weights_f_name.replace('/', '__')}.pkl")
    
    if os.path.exists(bs_preds_path):
        print(f"Path {bs_preds_path} exists. Skipping.")
        continue

    bs_predictions = weights_to_raw_predictions(
        grid_name = grid_name,
        model_getter=model_getter,
        weights_path = os.path.join(MODELS_ROOT, weights_f_name),
        char_tokenizer=char_tokenizer,
        dataset=grid_name_to_val_dataset[grid_name],
        generator_ctor=BeamGenerator,
        n_workers=4,
        generator_kwargs=generator_kwargs
    )

    with open(bs_preds_path, 'wb') as f:
        pickle.dump(bs_predictions, f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 9416/9416 [41:23<00:00,  3.79it/s] 


In [375]:
preds_name = "m1_bigger__m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt.pkl"
bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                preds_name)
with open(bs_preds_path, 'rb') as f:
    default_valid_preds_bs = pickle.load(f)

default_valid_preds_bs = patch_wrong_prediction_shape(default_valid_preds_bs)
default_valid_preds_bs = remove_beamsearch_probs(default_valid_preds_bs)
default_valid_preds_bs, _ = separate_out_vocab_all_crvs(default_valid_preds_bs, vocab_set)
get_mmr(default_valid_preds_bs, val_default_targets)

0.8929800339847141

In [340]:
preds_name = "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl"
bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                preds_name)
with open(bs_preds_path, 'rb') as f:
    extra_valid_preds_bs = pickle.load(f)

extra_valid_preds_bs = patch_wrong_prediction_shape(extra_valid_preds_bs)
extra_valid_preds_bs = remove_beamsearch_probs(extra_valid_preds_bs)
extra_valid_preds_bs, _ = separate_out_vocab_all_crvs(extra_valid_preds_bs, vocab_set)
get_mmr(extra_valid_preds_bs, val_extra_targets)

0.8876027397260277

In [ ]:
# {
#     "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl": 0.8811533559898118,
#     "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl": 0.8835960067969484,
#     "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl": 0.8900881478334822,
#     "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl": 0.8871590909090984,
#     "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl": 0.887674171622777,
#     "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl": 0.8877740016992428,
    
    
#     "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl": 0.8864383561643838,
#     "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl": 0.8876027397260277
    
# }

In [37]:
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt.pkl", #: 0.8929800339847141,
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl", #: 0.8914698385726496,
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",  #: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",  #: E 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",  #: 0.887674171622777,
        # "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",  #: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2__2023_11_12__17_40_42__0.30909_default_l2_1e-05_ls0.02_switch_0.pt.pkl",  # 0.8849384027187835
        # "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",  #: 0.8835960067969484,
        # "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",  #: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}

# Отранжированы по качесту beamsearch на валидации

In [ ]:
import pickle

default_idxs = val_default_dataset.grid_name_idxs
extra_idxs = val_extra_dataset.grid_name_idxs 

grid_name_to_augmented_preds = {}

for grid_name in ('default', 'extra'):
    bs_pred_list = []

    for f_name in grid_name_to_ranged_bs_model_preds_paths[grid_name]:
        f_path = os.path.join("../data/saved_beamsearch_validation_results/", f_name)
        with open(f_path, 'rb') as f:
            bs_pred_list.append(pickle.load(f))
        
    bs_pred_list = [patch_wrong_prediction_shape(bs_preds) for bs_preds in bs_pred_list] 
    bs_pred_list = [remove_beamsearch_probs(bs_preds) for bs_preds in bs_pred_list]
    bs_pred_list = [separate_out_vocab_all_crvs(bs_preds, vocab_set)[0] for bs_preds in bs_pred_list]


    augmented_preds = bs_pred_list.pop(0)

    while bs_pred_list:
        augmented_preds = append_preds(augmented_preds, bs_pred_list.pop(0))

    grid_name_to_augmented_preds[grid_name] = augmented_preds


full_preds = merge_default_and_extra_preds(
    grid_name_to_augmented_preds['default'],
    grid_name_to_augmented_preds['extra'],
    default_idxs,
    extra_idxs)

In [ ]:
full_preds[:10]

In [ ]:
from collections import defaultdict

n_preds_in_line_dict = defaultdict(int)

for line in full_preds:
    n_preds_in_line_dict[len(line)] += 1

print(n_preds_in_line_dict)

In [ ]:
all_targets = None
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding='utf-8') as f:
    all_targets = f.read().splitlines() 

In [ ]:
get_mmr(full_preds, all_targets)

```python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}
```

0.8936010000000082


``` python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}
```

0.892801000000009


``` python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

0.8936000000000084

## Beamsearch test

In [343]:
MAX_WORD_LEN = 36

generator_kwargs = {
    'max_steps_n': MAX_WORD_LEN - 1,
    'return_hypotheses_n': 7,
    'beamsize': 6,
    'normalization_factor': 0.5,
    'verbose': False
}

In [344]:
grid_name_to_test_dataset = {
    'default': test_default_dataset,
    'extra': test_extra_dataset
}

In [356]:
import pickle

bs_params = [
    ("extra", get_m1_bigger_model, "m1_bigger/m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt"),

    # "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl": 0.8835960067969484,
    # "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl": 0.8871590909090984,
    # "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl": 0.8864383561643838,
]


for grid_name, model_getter, weights_f_name in bs_params:

    bs_preds_path = os.path.join("../data/saved_beamsearch_results/",
                                f"{weights_f_name.replace('/', '__')}.pkl")
    
    if os.path.exists(bs_preds_path):
        print(f"Path {bs_preds_path} exists. Skipping.")
        continue

    bs_predictions = weights_to_raw_predictions(
        grid_name = grid_name,
        model_getter=model_getter,
        weights_path = os.path.join(MODELS_ROOT, weights_f_name),
        char_tokenizer=char_tokenizer,
        dataset=grid_name_to_test_dataset[grid_name],
        generator_ctor=BeamGenerator,
        n_workers=4,
        generator_kwargs=generator_kwargs
    )

    with open(bs_preds_path, 'wb') as f:
        pickle.dump(bs_predictions, f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 627/627 [06:02<00:00,  1.73it/s]


In [161]:
# len(clean_default_test_predictions), sum(bool(el) for el in clean_default_test_predictions)

(9373, 9193)

In [78]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [169]:
# from collections import defaultdict

# n_preds_in_line_dict = defaultdict(int)

# for line in clean_test_predictions:
#     n_preds_in_line_dict[len(line)] += 1

# print(n_preds_in_line_dict)

defaultdict(<class 'int'>, {4: 7404, 3: 1032, 2: 840, 1: 577, 0: 147})


In [170]:
old_preds_path = os.path.join(DATA_ROOT, "test_raw_pred___best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt__best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt.pkl")
with open(old_preds_path, 'rb') as f:
    old_preds_list = pickle.load(f)

In [171]:
old_preds_list = remove_beamsearch_probs(old_preds_list)

In [172]:
old_preds_list_valid, old_preds_list_invalid = separate_out_vocab_all_crvs(old_preds_list, vocab_set)

In [176]:
# submission_name = "default__m1_bigger_13679__m1_v2__14229___extra__14301___with_baseline__beam.csv"
# out_path = rf"..\data\submissions\{submission_name}"
# create_submission(clean_test_baseline_augmented, out_path)

# Submission creation

id 1

```
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': ["m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl"]
}
```

id 2

```
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

id 3

```python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger_m1_bigger_v2_2023_11_12_14_51_49_0_13115_greed_acc_0_86034.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

In [385]:
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger_m1_bigger_v2_2023_11_12_14_51_49_0_13115_greed_acc_0_86034.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации

In [386]:
default_idxs = test_default_dataset.grid_name_idxs
extra_idxs = test_extra_dataset.grid_name_idxs 

grid_name_to_augmented_preds = {}

for grid_name in ('default', 'extra'):
    bs_pred_list = []

    for f_name in grid_name_to_ranged_bs_model_preds_paths[grid_name]:
        f_path = os.path.join("../data/saved_beamsearch_results/", f_name)
        with open(f_path, 'rb') as f:
            bs_pred_list.append(pickle.load(f))
        
    bs_pred_list = [patch_wrong_prediction_shape(bs_preds) for bs_preds in bs_pred_list] 
    bs_pred_list = [remove_beamsearch_probs(bs_preds) for bs_preds in bs_pred_list]
    bs_pred_list = [separate_out_vocab_all_crvs(bs_preds, vocab_set)[0] for bs_preds in bs_pred_list]


    augmented_preds = bs_pred_list.pop(0)

    while bs_pred_list:
        augmented_preds = append_preds(augmented_preds, bs_pred_list.pop(0))

    grid_name_to_augmented_preds[grid_name] = augmented_preds


full_preds = merge_default_and_extra_preds(
    grid_name_to_augmented_preds['default'],
    grid_name_to_augmented_preds['extra'],
    default_idxs,
    extra_idxs)

In [387]:
from collections import defaultdict

n_preds_in_line_dict = defaultdict(int)

for line in full_preds:
    n_preds_in_line_dict[len(line)] += 1

print(n_preds_in_line_dict)

defaultdict(<class 'int'>, {4: 8188, 3: 706, 2: 606, 1: 407, 0: 93})


In [388]:
baseline_preds = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    baseline_preds = f.read().splitlines()
baseline_preds = [line.split(",") for line in augment_lines]

In [389]:
full_preds = append_preds(full_preds, baseline_preds)

In [390]:
create_submission(full_preds,
                  f"../data/submissions/id3_with_baseline_without_old_preds")

In [363]:
full_preds_augmentations = [
    
]